This is just to keep notes about the `predictors.py` file, to try to clearly present its structure.

`class` `DataTrainer`

Given comments:
* Base class for most "trainers", classes which take data and train themselves (fit a statistical model) to the data.
* Can also be used as a base for ("predictor") classes that can directly return a "prediction"

Properties:
* `data` = an instance of the `TimedPoints` class, which is the data to be trained on

Functions:
* `data(value)` = set the data


`class` `GridPrediction` inheriting from `data.BoundedGrid`

Given comments:
* This is a prediction based on a grid.
* The risk is computed by finding the grid cell containing the coordinates, then referring to the abstract `grid_risk` method.
* The "extent" of the prediction "is not meaningfully defined"

Initialization parameters:
* `xsize` = width of each grid cell
* `ysize` = height of each grid cell
* `xoffset` = how much to offset the input x coordinate by, default 0
* `yoffset` = how much to offset the input y coordinate by, default 0

Properties:
* `xextent` = ? (return 0)
* `yextent` = ? (return 0)
* `intensity_matrix` = a matrix that represents the risk. By default, this is generated as an array with dimensions `yextent` x `xextent`, with the risk calculated by `grid_risk` at every pair of coordinates from 0 to those limits. Also check whether any coordinates are "masked" and return such a masked array if necessary.

Functions:
* `risk(x,y)` = return the estimated risk at coordinate (x,y). Performs this by calling its own `grid_risk` function on the given coordinates.
* `grid_risk(gridx,gridy)` = return the estimated risk at coordinates (gridx, gridy). Not directly implemented here, but its declaration means that classes that inherit from this class are required to define such a function.
* `is_valid(gx,gy)` = return whether the grid cell is "included in the possibly masked grid"

`class` `GridPredictionArray` inherited from `GridPrediction`

Given comments:
* This is `GridPrediction` backed by a numpy array

Initialization parameters:
* `xsize` = width of each grid cell
* `ysize` = height of each grid cell
* `matrix` = 2d numpy array
* `xoffset` = how much to offset the input x coordinate by, default 0
* `yoffset` = how much to offset the input y coordinate by, default 0

Properties:
* `intensity_matrix` = the `matrix` of risk values
* `xextent` = the width of the `matrix`
* `yextent` = the height of the `matrix`

Functions:
* `clone()` = return new instance, with same data
* `new_extent()` = return a new instance with different offsets and extents, which also shifts the risk matrix appropriately as well, though new cells will have intensity zero. If a mask matrix exists, it is not preserved.
* `zero_to_constant()` = if `matrix` is constantly zero, change it to be the same size but constantly ones.
     * Not sure if it's implemented correctly? Seems to check whether the matrix's sum is 0 or less, not whether the matrix is constantly 0. Also there might be better implementations like ".zeroslike" instead, maybe?
* `renormalise()` = divide values of `matrix` by its sum in order to normalise the matrix
* `break_ties()` = return a new `GridPredictionArray` after adding a small amount of noise to cells with identical intensity values so they're no longer perfectly identical
    * I haven't analyzed the code to check it works right
* `grid_risk(gx,gy)` = obtain the risk for a particular cell at coordinates (gx,gy).
    * This really doesn't look right because it seems like it always returns `out` but then there's more code after those lines? I also can't really tell what data type `gx` and `gy` are supposed to be, and what `mask` is...
* `from_continuous_prediction(prediction, width, height)` = construct instance of `GridPredictionArray` from an instance of `ContinuousPrediction` by using its grid size and offset. First, create `matrix` by using the `to_matrix` function on the `prediction`, then create `GridPredictionArray` with the cell width & height of the prediction, that `matrix`, and the prediction's offsets.
* `from_continuous_prediction_region(prediction, region, cell_width, cell_height)` = construct instance of `GridPredictionArray` from an instance of `ContinuousPrediction` by using the above function `from_continuous_prediction`, but also uses `region` of type `RectangularRegion` to specify the area under consideration.
    * Note that in the code, `width` and `height` correspond to the number of cells per row and column calculated as the dimensions of the region divided by the cell width and height.
    * The code also uses the function `rebase` on the prediction in order to limit its scope to that of the specified `region`; it is this new prediction `newpred` that is passed into `from_continuous_prediction`.
* `from_continuous_prediction_grid(prediction, grid)` = construct instance of `GridPredictionArray` from an instance of `ContinuousPrediction` by using the above function `from_continuous_prediction`, but also uses `grid` of type `BoundedGrid` to base the grid on. So it's similar to `from_continuous_prediction_region` but when the `prediction` is `rebase`'d into `newpred` you use the dimensions and offsets from the `grid` instead of a `region`. 
* `is_valid(gx,gy)` = check if the matrix at the given coordinates is not masked
* `mesh_data()` = return pair (xcoords, ycoords) that can be used with function `intensity_matrix` to be passed to `matplotlib.pcolor`.
    * `intensity_matrix[i][j]` is the risk intensity in the cell defined by opposite vertices `(xcoords[j], ycoords[i])` and `(xcoords[j+1], ycoords[i+1])`
    * created via clever combination of functions: for each dimension, make a range of size corresponding to matrix dimension, multiply by (x|y)size, then add (x|y)offset
    * wait, is that clever? instead of multiplying and adding, can't that be folded into the range function directly?
* `percentile_matrix()` = return matrix same shape as `intensity_matrix` but where the float values give the percentile of the risk, normalised to [0,1]. Cell with highest risk is assigned 1.0. Ties are rounded up.
    * Haven't confirmed code is correct
* `mask_with(mask)` = mask the intensity matrix with an instance of class `MaskedGrid` that is of matching size and offset

`class` `ContinuousPrediction`

Given comments:
* A prediction that allows the risk to be calculated at any point in a continuous fashion.
* Allows Monte Carlo sampling to produce a grid risk

Initialization parameters:
* `cell_width` = width of internal cells
* `cell_height` = height of internal cells
* `xoffset` = x coordinate of the start of the grid
* `yoffset` = y coordinate of the start of the grid
* `samples` = number of samples to use when computing the risk in a grid cell
    * "set to `None` to use a fixed density" (?)
    * "set to a negative number to sample on a regular pattern" (?)

Properties:
* `samples` = number of samples to use per cell.
    * if `None` then uses fixed density, which is hard-coded as 1 sample per 200 units of area (see `samples(v)` function)
    * if negative, say -x, then sample on a subgrid of size x\*x per cell, to be more reproducible (??? - Does it sample at corner of cells? Centers of cells? A random point within each cell? - ???)

Functions:
* `samples(v)` = set `samples` to be the value v.
    * if v is `None` then set `samples` to be `cell_width * cell_height / 200`, but a minimum of 2
* `grid_risk(gx,gy)` = return an estimate of the average risk in the defined grid cell
    * Assumes `samples` is non-negative
    * Generates `samples` number of random points within the given cell
    * (gx=2,gy=7) would mean the cell in the 3rd column, 8th row, for example
    * runs `risk(x,y)` where x and y are lists/arrays of x and y coordinates, then takes average
* `risk_array(x,y)` = like `risk(x,y)` but in blocks of at most 50, "to avoid excessive memory usage"
    * check x is a 1d array, get its length
    * for each iteration of while loop, n (offset) is either 0 (if first iter), previous n+50 (for mid iters), or length (if last iter);
    * 
    * 
* `to_matrix(width, height)` = sample the risk at each grid point from (0,0) to (w-1,h-1)
    * if `samples` is negative, run `to_matrix_grid(width, height)`
    * else, for each row of cells, 